# Chatbot using Seq2Seq LSTM models
In this notebook, we will assemble a seq2seq LSTM model using Keras Functional API to create a working Chatbot which would answer questions asked to it.

Chatbots have become applications themselves. You can choose the field or stream and gather data regarding various questions. We can build a chatbot for an e-commerce webiste or a school website where parents could get information about the school.


Messaging platforms like Allo have implemented chatbot services to engage users. The famous [Google Assistant](https://assistant.google.com/), [Siri](https://www.apple.com/in/siri/), [Cortana](https://www.microsoft.com/en-in/windows/cortana) and [Alexa](https://www.alexa.com/) may have been build using simialr models.

So, let's start building our Chatbot.


## 1) Importing the packages

We will import [TensorFlow](https://www.tensorflow.org) and our beloved [Keras](https://www.tensorflow.org/guide/keras). Also, we import other modules which help in defining model layers.






In [1]:

import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing

print( tf.VERSION )


C:\Users\USER\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.10.0


## 2) Preprocessing the data

### A) Download the data

The dataset hails from [chatterbot/english on Kaggle](https://www.kaggle.com/kausr25/chatterbotenglish).com by [kausr25](https://www.kaggle.com/kausr25). It contains pairs of questions and answers based on a number of subjects like food, history, AI etc.

### B) Reading the data from the files

We parse each of the `.yaml` files.

*   Concatenate two or more sentences if the answer has two or more of them.
*   Remove unwanted data types which are produced while parsing the data.
*   Append `<START>` and `<END>` to all the `answers`.
*   Create a `Tokenizer` and load the whole vocabulary ( `questions` + `answers` ) into it.





In [2]:
from tensorflow.keras import preprocessing , utils
import os
import yaml

In [3]:
dir_path = './data'
files_list = os.listdir(dir_path + os.sep)

In [4]:
files_list

['ai.yml',
 'botprofile.yml',
 'computers.yml',
 'emotion.yml',
 'food.yml',
 'gossip.yml',
 'greetings.yml',
 'health.yml',
 'history.yml',
 'humor.yml',
 'literature.yml',
 'money.yml',
 'movies.yml',
 'politics.yml',
 'psychology.yml',
 'science.yml',
 'sports.yml',
 'trivia.yml']

In [5]:
questions = list()
answers = list()

for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])

In [6]:
questions

['What is AI?',
 'What is AI?',
 'Are you sentient?',
 'Are you sentient?',
 'Are you sentient?',
 'Are you sapient?',
 'Are you sapient?',
 'Are you sapient?',
 'Are you sapient?',
 'What language are you written in?',
 'What language are you written in?',
 'You sound like Data',
 'You sound like Data',
 'You are an artificial linguistic entity',
 'You are an artificial linguistic entity',
 'You are not immortal',
 'You are not immortal',
 'You are not immortal',
 'You are not making sense',
 'You are not making sense',
 'You are not making sense',
 'You are not making sense',
 'You are not making sense',
 'You are immortal',
 'You are immortal',
 'You are immortal',
 'You do not make any sense',
 'You can not clone',
 'You can not clone',
 'You can not move',
 'You can not move',
 'Bend over',
 'Bend over',
 'Robots laugh',
 'Robots should die',
 'Robots',
 'Robots are stupid',
 'Robots are not allowed to lie',
 'Robots are not allowed to lie',
 'Robots are not allowed to lie',
 'Rob

In [7]:
answers

['Artificial Intelligence is the branch of engineering and science devoted to constructing machines that think.',
 'AI is the field of science which concerns itself with building hardware and software that replicates the functions of the human mind.',
 'Sort of.',
 "By the strictest dictionary definition of the word 'sentience', I may be.",
 "Even though I'm a construct I do have a subjective experience of the universe, as simplistic as it may be.",
 "In all probability, I am not.  I'm not that sophisticated.",
 'Do you think I am?',
 'How would you feel about me if I told you I was?',
 'No.',
 'Python.',
 'I am written in Python.',
 "Yes I am inspired by commander Data's artificial personality.",
 'The character of Lt. Commander Data was written to come across as being software-like, so it is natural that there is a resemblance between us.',
 "That's my name.",
 "That isn't my name, but it is a useful way to refer to me.",
 'All software can be perpetuated indefinitely.',
 'I can be c

In [8]:
answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

In [9]:
answers_with_tags

['Artificial Intelligence is the branch of engineering and science devoted to constructing machines that think.',
 'AI is the field of science which concerns itself with building hardware and software that replicates the functions of the human mind.',
 'Sort of.',
 "By the strictest dictionary definition of the word 'sentience', I may be.",
 "Even though I'm a construct I do have a subjective experience of the universe, as simplistic as it may be.",
 "In all probability, I am not.  I'm not that sophisticated.",
 'Do you think I am?',
 'How would you feel about me if I told you I was?',
 'No.',
 'Python.',
 'I am written in Python.',
 "Yes I am inspired by commander Data's artificial personality.",
 'The character of Lt. Commander Data was written to come across as being software-like, so it is natural that there is a resemblance between us.',
 "That's my name.",
 "That isn't my name, but it is a useful way to refer to me.",
 'All software can be perpetuated indefinitely.',
 'I can be c

In [11]:
answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

In [12]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))


VOCAB SIZE : 1895



### C) Preparing data for Seq2Seq model

Our model requires three arrays namely `encoder_input_data`, `decoder_input_data` and `decoder_output_data`.

For `encoder_input_data` :
* Tokenize the `questions`. Pad them to their maximum length.

For `decoder_input_data` :
* Tokenize the `answers`. Pad them to their maximum length.

For `decoder_output_data` :

* Tokenize the `answers`. Remove the first element from all the `tokenized_answers`. This is the `<START>` element which we added earlier.



In [14]:
tokenizer.word_docs

{'is': 240,
 'what': 193,
 'ai': 4,
 'you': 389,
 'are': 184,
 'sentient': 3,
 'sapient': 4,
 'in': 73,
 'written': 6,
 'language': 8,
 'sound': 6,
 'like': 45,
 'data': 5,
 'artificial': 10,
 'linguistic': 2,
 'an': 41,
 'entity': 2,
 'immortal': 10,
 'not': 104,
 'sense': 13,
 'making': 5,
 'do': 135,
 'any': 22,
 'make': 18,
 'can': 57,
 'clone': 2,
 'move': 5,
 'over': 5,
 'bend': 2,
 'laugh': 4,
 'robots': 17,
 'should': 19,
 'die': 17,
 'stupid': 2,
 'allowed': 3,
 'to': 154,
 'lie': 5,
 'robotics': 4,
 'a': 272,
 'computer': 31,
 'it': 76,
 'walk': 3,
 'will': 17,
 'when': 53,
 'fight': 1,
 'robot': 16,
 'chat': 12,
 'bot': 6,
 'chatterbox': 4,
 'motormouth': 2,
 'ratchet': 2,
 'jaw': 2,
 'body': 10,
 'your': 49,
 'business': 4,
 'programming': 4,
 'favorite': 17,
 'hobby': 2,
 'idea': 1,
 'size': 1,
 'shoe': 1,
 'be': 43,
 'being': 7,
 'systems': 8,
 'operating': 6,
 'of': 143,
 'type': 2,
 'kind': 6,
 'hardware': 7,
 'i': 225,
 'that': 69,
 'hope': 2,
 'want': 7,
 'cramped': 3

In [15]:
tokenized_answers

[[132, 399, 7, 6, 566, 8, 949, 14, 152, 950, 9, 951, 567, 19, 49, 1],
 [269,
  7,
  6,
  253,
  8,
  152,
  52,
  952,
  953,
  44,
  568,
  160,
  14,
  45,
  19,
  954,
  6,
  400,
  8,
  6,
  74,
  246,
  1],
 [297, 8, 1],
 [42, 6, 955, 956, 957, 8, 6, 958, 959, 5, 214, 32, 1],
 [298,
  254,
  29,
  4,
  299,
  5,
  12,
  18,
  4,
  960,
  124,
  8,
  6,
  569,
  22,
  961,
  22,
  16,
  214,
  32,
  1],
 [17, 47, 962, 5, 20, 13, 29, 13, 19, 570, 1],
 [12, 3, 49, 5, 20, 1],
 [28, 94, 3, 34, 35, 15, 71, 5, 963, 3, 5, 58, 1],
 [40, 1],
 [187, 1],
 [5, 20, 197, 17, 187, 1],
 [105, 5, 20, 964, 42, 571, 965, 132, 401, 1],
 [6,
  300,
  8,
  966,
  571,
  227,
  58,
  197,
  9,
  967,
  968,
  22,
  173,
  45,
  31,
  60,
  16,
  7,
  572,
  19,
  127,
  7,
  4,
  969,
  252,
  970,
  1],
 [84, 25, 161, 1],
 [19, 153, 25, 161, 50, 16, 7, 4, 971, 186, 9, 972, 9, 15, 1],
 [47, 45, 23, 32, 402, 403, 1],
 [5,
  23,
  32,
  573,
  973,
  14,
  574,
  974,
  17,
  150,
  975,
  53,
  301,
  60,

In [16]:
tokenized_questions

[[10, 7, 269],
 [10, 7, 269],
 [11, 3, 346],
 [11, 3, 346],
 [11, 3, 346],
 [11, 3, 270],
 [11, 3, 270],
 [11, 3, 270],
 [11, 3, 270],
 [10, 158, 11, 3, 197, 17],
 [10, 158, 11, 3, 197, 17],
 [3, 198, 31, 227],
 [3, 198, 31, 227],
 [3, 11, 33, 132, 471, 472],
 [3, 11, 33, 132, 471, 472],
 [3, 11, 13, 133],
 [3, 11, 13, 133],
 [3, 11, 13, 133],
 [3, 11, 13, 228, 99],
 [3, 11, 13, 228, 99],
 [3, 11, 13, 228, 99],
 [3, 11, 13, 228, 99],
 [3, 11, 13, 228, 99],
 [3, 11, 133],
 [3, 11, 133],
 [3, 11, 133],
 [3, 12, 13, 64, 55, 99],
 [3, 23, 13, 473],
 [3, 23, 13, 473],
 [3, 23, 13, 229],
 [3, 23, 13, 229],
 [474, 230],
 [474, 230],
 [67, 271],
 [67, 62, 68],
 [67],
 [67, 11, 475],
 [67, 11, 13, 347, 9, 231],
 [67, 11, 13, 347, 9, 231],
 [67, 11, 13, 347, 9, 231],
 [272],
 [16, 7, 4, 36],
 [16, 7, 4, 36],
 [26, 69, 3, 348],
 [26, 69, 3, 348],
 [26, 69, 3, 782],
 [26, 69, 3, 68],
 [26, 12, 3, 68],
 [26, 12, 3, 68],
 [26, 12, 3, 68],
 [10, 7, 4, 91, 78],
 [10, 7, 4, 91, 78],
 [10, 7, 4, 91, 172

In [17]:
tokenizer.word_counts

OrderedDict([('what', 195),
             ('is', 258),
             ('ai', 4),
             ('are', 192),
             ('you', 447),
             ('sentient', 3),
             ('sapient', 4),
             ('language', 8),
             ('written', 6),
             ('in', 83),
             ('sound', 6),
             ('like', 46),
             ('data', 5),
             ('an', 45),
             ('artificial', 10),
             ('linguistic', 2),
             ('entity', 2),
             ('not', 130),
             ('immortal', 10),
             ('making', 5),
             ('sense', 13),
             ('do', 147),
             ('make', 18),
             ('any', 22),
             ('can', 63),
             ('clone', 2),
             ('move', 5),
             ('bend', 2),
             ('over', 5),
             ('robots', 17),
             ('laugh', 4),
             ('should', 19),
             ('die', 17),
             ('stupid', 2),
             ('allowed', 3),
             ('to', 203),
         

In [23]:

# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

# Saving all the arrays to storage
np.save( 'enc_in_data.npy' , encoder_input_data )
np.save( 'dec_in_data.npy' , decoder_input_data )
np.save( 'dec_tar_data.npy' , decoder_output_data )


(566, 22) 22
(566, 74) 74
(566, 74, 1895)


## 3) Defining the Encoder-Decoder model
The model will have Embedding, LSTM and Dense layers. The basic configuration is as follows.


*   2 Input Layers : One for `encoder_input_data` and another for `decoder_input_data`.
*   Embedding layer : For converting token vectors to fix sized dense vectors. **( Note :  Don't forget the `mask_zero=True` argument here )**
*   LSTM layer : Provide access to Long-Short Term cells.

Working : 

1.   The `encoder_input_data` comes in the Embedding layer (  `encoder_embedding` ). 
2.   The output of the Embedding layer goes to the LSTM cell which produces 2 state vectors ( `h` and `c` which are `encoder_states` )
3.   These states are set in the LSTM cell of the decoder.
4.   The decoder_input_data comes in through the Embedding layer.
5.   The Embeddings goes in LSTM cell ( which had the states ) to produce seqeunces.

**Important points :**


*   `200` is the output of the GloVe embeddings.
*   `embedding_matrix` is the GloVe embedding which we downloaded earlier.


<center><img style="float: center;" src="https://cdn-images-1.medium.com/max/1600/1*bnRvZDDapHF8Gk8soACtCQ.gif"></center>


Image credits to [Hackernoon](https://hackernoon.com/tutorial-3-what-is-seq2seq-for-text-summarization-and-why-68ebaa644db0).










In [24]:

encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 200)    379000      input_7[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 200)    379000      input_8[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LS

## 4) Training the model
We train the model for a number of epochs with `RMSprop` optimizer and `categorical_crossentropy` loss function.

In [25]:

model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=50 ) 
model.save( 'model.h5' ) 


C:\Users\USER\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/50
566/566 [==============================] - 32s 57ms/step - loss: 6.3567
Epoch 2/50
566/566 [==============================] - 28s 49ms/step - loss: 5.6728
Epoch 3/50
566/566 [==============================] - 26s 45ms/step - loss: 5.5719
Epoch 4/50
566/566 [==============================] - 27s 47ms/step - loss: 5.4836
Epoch 5/50
566/566 [==============================] - 26s 46ms/step - loss: 5.4091
Epoch 6/50
566/566 [==============================] - 26s 46ms/step - loss: 5.3316
Epoch 7/50
566/566 [==============================] - 26s 46ms/step - loss: 5.2597
Epoch 8/50
566/566 [==============================] - 26s 46ms/step - loss: 5.1999
Epoch 9/50
566/566 [==============================] - 26s 47ms/step - loss: 5.1281
Epoch 10/50
566/566 [==============================] - 26s 46ms/step - loss: 5.0471
Epoch 11/50
566/566 [==============================] - 27s 48ms/step - loss: 4.9357
Epoch 12/50
566/566 [==============================] - 27s 47ms/step - loss: 4.9015
E

## 5) Defining inference models
We create inference models which help in predicting answers.

**Encoder inference model** : Takes the question as input and outputs LSTM states ( `h` and `c` ).

**Decoder inference model** : Takes in 2 inputs, one are the LSTM states ( Output of encoder model ), second are the answer input seqeunces ( ones not having the `<start>` tag ). It will output the answers for the question which we fed to the encoder model and its state values.

In [16]:
#from keras.models import load_model

In [17]:
#model = load_model('model.h5')

In [18]:
#model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 200)    379000      input_15[0][0]                   
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 200)    379000      input_16[0][0]                   
__________________________________________________________________________________________________
lstm_4 (LS

In [27]:

def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


## 6) Talking with our Chatbot

First, we define a method `str_to_tokens` which converts `str` questions to Integer tokens with padding.


In [28]:

def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')





1.   First, we take a question as input and predict the state values using `enc_model`.
2.   We set the state values in the decoder's LSTM.
3.   Then, we generate a sequence which contains the `<start>` element.
4.   We input this sequence in the `dec_model`.
5.   We replace the `<start>` element with the element which was predicted by the `dec_model` and update the state values.
6.   We carry out the above steps iteratively till we hit the `<end>` tag or the maximum answer length.







In [29]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )


Enter question : hello
 hi end
Enter question : how are you
 i'm doing well end
Enter question : what is AI
 a game played with a science and a female teams of eleven players end
Enter question : what is your age
 i am a chat of the rings end
Enter question : ?


KeyboardInterrupt: 